In [37]:
from captcha.image import ImageCaptcha # 验证码生成库 （本代码用这个库）
import numpy as np
from PIL import Image # 处理图像的包
import random
import sys
import os
import tensorflow as tf
import matplotlib.pyplot as plt

In [38]:
number = [str(i) for i in range(10)]
little = [chr(i) for i in range(97,123)]
large = [chr(i) for i in range(65,91)]
print(number)
print(little)
print(large)
CHAR_SET = number # 数组

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']


##  生成验证码

如果报错：
AttributeError: module 'random' has no attribute 'choices'

问题：choices is added in version 3.6. Presumably the live server uses an older version while you locally use 3.6.

---

* 解决办法1：升级到python3.6

* 解决方法2： from numpy.random import choice
```
from numpy.random import choice

colors = choice(['red','green'],size=10)
print(colors)

```



In [43]:
def random_captcha_text(char_set=number,captcha_size=4):
    # return ''.join(random.choices(CHAR_SET,k=4)) # AttributeError: module 'random' has no attribute 'choices'
    from numpy.random import choice
    # print(random.choice(CHAR_SET,size=captcha_size))
    return ''.join(choice(CHAR_SET,size=captcha_size))

# 生成字符对应的验证码
def gen_captcha_text_and_image(path):
    image_ = ImageCaptcha() 
    # 获得随机生成的验证码
    captcha_text = random_captcha_text()
    # 生成验证码并保存为图片
    captcha = image_.generate(captcha_text)
    image_.write(captcha_text, path + captcha_text + '.jpg')

path = '10captcha/images/'
num = 100 # 验证码图片数量

if not os.path.exists(path):
    os.makedirs(path)
for i in range(num):
    gen_captcha_text_and_image(path)
    sys.stdout.write('\r>> Creating image %d/%d' % (i+1,num))
    sys.stdout.flush()
sys.stdout.write('\n')
sys.stdout.flush()

>> Creating image 100/100


## 生成tfrecord

In [48]:
_NUM_TEST = 500

_RANDOM_SEED = 2

# 数据集路径
DATA_DIR = '10captcha/images/'
# tfrecoder文件存放路径
TFRECORD_DIR = '10captcha/'

# 判断tfrecoder文件是否存在
def _tfdata_exists(dataset_dir):
    for split_name in ['train','test']:
        tf_name = os.path.join(dataset_dir,split_name + '.tfrecord')
        if not tf.gfile.Exists(tf_name):
            return False
    return True

# 获取所以验证码图片
def _get_filenames(dataset_dir):
    captcha_names = []
    for filename in os.listdir(dataset_dir):
        # 获取文件路径
        path = os.path.join(dataset_dir, filename)
        captcha_names.append(path)
    return captcha_names

def int64_feature(values):
    if not isinstance(values, (tuple, list)):
        values = [values]
    return tf.train.Feature(int64_list=tf.train.Int64List(value=values))

def bytes_feature(values):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value = [values]))

def image_to_tfexample(image_data,label0,label1,label2,label3):
    return tf.train.Example(features=tf.train.Features(feature={
     'image': bytes_feature(image_data),
        'label0':int64_feature(label0),
        'label1':int64_feature(label1),
        'label2':int64_feature(label2),
        'label3':int64_feature(label3)
    }))

# 数据转换为record格式
def _convert_dataset(split_name, filenames, tfrecord_dir):
    assert split_name in ['train','test']
    
    with tf.Session() as sess:
        out_tfnames = os.path.join(tfrecord_dir, split_name + '.tfrecord')
        with tf.python_io.TFRecordWriter(out_tfnames) as tf_writer:
            for i,filename in enumerate(filenames):
                try:
                    sys.stdout.write('\r>> Converting image %d/%d' % (i+1,len(filenames)))
                    sys.stdout.flush()
                    
                    image_data = Image.open(filename)
                    image_data = image_data.resize((224,224))
                    image_data = image_data.convert('L').tobytes()
                    
                    # 拿到4位验证码并encode()
                    labels = filename.split('/')[-1][:4]
                    
                    example = image_to_tfexample(image_data, int(labels[0]), int(labels[1]), int(labels[2]), int(labels[3]))
                    tf_writer.write(example.SerializeToString())
                
                except IOError as e:
                    print('Wrong: ' + filename)
                    print('Error: ',e)
                    print('Skip it\n')
    sys.stdout.write('\n')
    sys.stdout.flush()
# 判断tfrecoder文件是否存在
if _tfdata_exists(TFRECORD_DIR):
    print('tf文件已存在')
else:
    # 获得所有照片
    photo_filenames = _get_filenames(DATA_DIR)
    
    random.seed(_RANDOM_SEED)
    # 把数据集企鹅分为训练集和测试集 打乱文件
    random.shuffle(photo_filenames)
    training_filenames = photo_filenames[_NUM_TEST:]
    testing_filenames = photo_filenames[:_NUM_TEST]
    # 数据转换
    _convert_dataset('train', training_filenames, TFRECORD_DIR)
    _convert_dataset('test', testing_filenames, TFRECORD_DIR)


>> Converting image 98/98
